In [37]:
from collections import deque
import re

sample="""Sensor at x=2, y=18: closest beacon is at x=-2, y=15
Sensor at x=9, y=16: closest beacon is at x=10, y=16
Sensor at x=13, y=2: closest beacon is at x=15, y=3
Sensor at x=12, y=14: closest beacon is at x=10, y=16
Sensor at x=10, y=20: closest beacon is at x=10, y=16
Sensor at x=14, y=17: closest beacon is at x=10, y=16
Sensor at x=8, y=7: closest beacon is at x=2, y=10
Sensor at x=2, y=0: closest beacon is at x=2, y=10
Sensor at x=0, y=11: closest beacon is at x=2, y=10
Sensor at x=20, y=14: closest beacon is at x=25, y=17
Sensor at x=17, y=20: closest beacon is at x=21, y=22
Sensor at x=16, y=7: closest beacon is at x=15, y=3
Sensor at x=14, y=3: closest beacon is at x=15, y=3
Sensor at x=20, y=1: closest beacon is at x=15, y=3""".splitlines()

def distance(a, b): return abs(a[0] - b[0]) + abs(a[1] - b[1])

def parse(lines:list[str]):
    S = set()
    B = set()
    S_to_B = {}
    S_dist = {}
    for line in lines:
        [res] = re.findall("Sensor at x=(.+), y=(.+): closest beacon is at x=(.+), y=(.+)", line)
        [sx,sy,bx,by] = map(int,res)
        s = (sx, sy)
        b = (bx, by)
        S.add(s)
        B.add(b)
        S_to_B[s] = b
        S_dist[s] = distance(s,b)
    return S, B, S_to_B, S_dist

def merge_segments(segments:list[tuple[int,int]]):
    segments.sort(key=lambda a:a[0])
    i = 0
    while i < len(segments)-1:
        start, end = segments[i]
        next_start, next_end = segments[i+1]
        if end+1 < next_start or start-1 > next_end: 
            i += 1
            continue
        segments[i] = (min(start,next_start), max(end, next_end))
        segments.remove(segments[i+1])
    return segments

def test(segments, outcome):
    merge_segments(segments)
    if set(segments) != set(outcome):
        print(f"Mismatch: {segments} != {outcome}")
    assert set(segments) == set(outcome)


test([(1,2),(4,5)], [(1,2), (4,5)])
test([(1,2),(3,5)], [(1,5)])
test([(1,3),(2,5)], [(1,5)])
test([(2,3),(2,5)], [(2,5)])
test([(1,1),(2,2),(3,3),(4,4),(5,5)], [(1,5)])

def segments_at(y,S,S_dist):
    segments = []
    for s in S:
        d = S_dist[s]
        sx,sy = s
        if y < sy - d or y > sy + d: continue

        width = d - abs(sy-y)
        segments.append((sx - width, sx + width))
    return segments

def part1(lines, y):
    S, B, S_to_B, S_dist = parse(lines)
    segments = merge_segments(segments_at(y, S, S_dist))
    width = sum(b-a+1 for a,b in segments)
    bs = sum(b==y for a,b in B)
    ss = sum(b==y for a,b in S)
    return width - bs - ss

def part2(lines):
    S, B, S_to_B, S_dist = parse(lines)

    at = lambda y: merge_segments(segments_at(y, S, S_dist))

    prev_segments = at(0)
    for y in range(1,4000000):
        segments = at(y)
        #print(f"{y=} {len(segments)=} {len(prev_segments)=}")
        if len(segments)>len(prev_segments):
            break
        segments = prev_segments
    print(y)

    return y + (segments[0][1] + 1) * 4000000

assert 26 == part1(sample, 10)
assert 56000011 == part2(sample)


11


In [39]:
from aoc import read_lines

lines = read_lines("data/day15.txt")
print(part1(lines, 2000000))

print(part2(lines))

6275922
3442119
11747175442119
